# Using Langchain in Gemini

[Tutorial on Youtube](https://youtu.be/aywZrzNaKjs)


In [4]:
import os
import sys

from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

sys.path.append(os.path.abspath(os.pardir))
load_dotenv()

True

In [3]:
# GoogleGenerativeAI(not ChatGoogleGenerativeAI) does not support system messages!
llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True, max_output_tokens=800)

messages = [
    SystemMessage(content="You are an expert data scientist."),
    HumanMessage(content="Please tell me who are you in a brief sentence.")
]

result = llm.invoke(messages)
result

AIMessage(content='I am a highly skilled professional with a deep understanding of data analysis, machine learning algorithms, and statistical methods, enabling me to extract insights from complex data.')

In [5]:
template = """
You are an expert data scientist with an expertise in building deep learning models.
Explain the concept of {concept} in a couple of lines.
"""
prompt = PromptTemplate(input_variables=['concept'], template=template)

In [6]:
prompt

PromptTemplate(input_variables=['concept'], template='\nYou are an expert data scientist with an expertise in building deep learning models.\nExplain the concept of {concept} in a couple of lines.\n')

In [8]:
llm.invoke(prompt.format(concept="autoencoder"))

AIMessage(content='An autoencoder is a type of artificial neural network that learns to compress and reconstruct data, allowing it to learn efficient representations of the input data.\nIt consists of an encoder network that maps the input data to a latent space and a decoder network that reconstructs the input data from the latent representation.')

In [10]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

chain.invoke("autoencoder")

{'concept': 'autoencoder',
 'text': 'An autoencoder is a type of deep learning model that learns to reconstruct its own input. It consists of an encoder part, which compresses the input into a latent representation, and a decoder part, which reconstructs the input from the latent representation.'}

In [12]:
second_prompt = PromptTemplate(
    input_variables=["ml_concept"],
    template="Turn the concept description of {ml_concept} and explain it to me like I'm five.")
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [14]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)
result = overall_chain.invoke("autoencoder")
result



> Entering new SimpleSequentialChain chain...
An autoencoder is a type of artificial neural network that learns to copy its input to its output. It consists of an encoder, which compresses the input into a lower-dimensional representation, and a decoder, which reconstructs the input from the compressed representation.
Imagine you have a magic machine that can take a picture of your favorite toy and turn it into a puzzle. The machine first breaks the picture into lots of tiny pieces, and then it mixes them all up. But here's the cool part: the machine can also put the pieces back together again and make the picture of your toy look just like it did before!

This magic machine is called an autoencoder. It has two parts: an encoder and a decoder. The encoder is like the part of the machine that breaks the picture into pieces. It takes the picture of your toy and turns it into a code that is much smaller than the original picture. The decoder is like the part of the machine that puts the

{'input': 'autoencoder',
 'output': "Imagine you have a magic machine that can take a picture of your favorite toy and turn it into a puzzle. The machine first breaks the picture into lots of tiny pieces, and then it mixes them all up. But here's the cool part: the machine can also put the pieces back together again and make the picture of your toy look just like it did before!\n\nThis magic machine is called an autoencoder. It has two parts: an encoder and a decoder. The encoder is like the part of the machine that breaks the picture into pieces. It takes the picture of your toy and turns it into a code that is much smaller than the original picture. The decoder is like the part of the machine that puts the pieces back together again. It takes the code from the encoder and turns it back into a picture of your toy.\n\nAutoencoders are used for lots of things, like helping computers to recognize objects and understand language. They can also be used to compress data, which means making 

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)

texts = text_splitter.create_documents([result["output"]])

In [17]:
texts

[Document(page_content='Imagine you have a magic machine that can take a picture of your favorite toy and turn it into a'),
 Document(page_content='puzzle. The machine first breaks the picture into lots of tiny pieces, and then it mixes them all'),
 Document(page_content="up. But here's the cool part: the machine can also put the pieces back together again and make the"),
 Document(page_content='picture of your toy look just like it did before!'),
 Document(page_content='This magic machine is called an autoencoder. It has two parts: an encoder and a decoder. The'),
 Document(page_content='encoder is like the part of the machine that breaks the picture into pieces. It takes the picture'),
 Document(page_content='of your toy and turns it into a code that is much smaller than the original picture. The decoder is'),
 Document(page_content='like the part of the machine that puts the pieces back together again. It takes the code from the'),
 Document(page_content='encoder and turns it back i

In [18]:
texts[0].page_content

'Imagine you have a magic machine that can take a picture of your favorite toy and turn it into a'

In [19]:
# langchain does not support gemini embeddings yet!

import google.generativeai as genai

genai.embed_content(model="models/embedding-001",
                    content=texts[0].page_content,
                    task_type="retrieval_document",
                    title="Embedding of single string")

{'embedding': [-0.032809824,
  -0.007542797,
  -0.026657125,
  0.0015110776,
  0.06376518,
  -0.016255857,
  0.064225525,
  -0.019796554,
  0.0115468055,
  0.0012582089,
  0.05256967,
  -0.009799506,
  -0.020394828,
  -0.039024737,
  -0.0059237205,
  -0.046774313,
  0.018347584,
  0.017060122,
  0.012180557,
  -0.05391766,
  0.029008852,
  0.00062611303,
  0.0026693214,
  -0.004493161,
  0.0039921096,
  0.028359002,
  0.07297254,
  -0.048622277,
  -0.009961648,
  0.050244883,
  -0.0095133325,
  0.014227774,
  -0.043162137,
  0.013286352,
  0.018672016,
  -0.07102841,
  -0.009505847,
  -0.019822018,
  -0.00214202,
  0.012558065,
  -0.011910497,
  -0.03412174,
  -0.017154858,
  -0.030536048,
  0.0023607484,
  0.011463036,
  -0.019538186,
  0.05276907,
  0.020636467,
  -0.09591086,
  0.030592214,
  0.007046663,
  0.08677839,
  -0.02805579,
  -0.026336359,
  -0.038983542,
  0.03649504,
  0.016716154,
  -0.004656456,
  -0.06169712,
  -0.012825988,
  -0.008992492,
  0.0019523746,
  0.0366164